<a href="https://colab.research.google.com/github/SSarvesh7206/Capstone_project_S.Sarvesh/blob/main/Model_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pathway bokeh --quiet

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

In [ ]:
data=pd.read_csv("/content/dataset.csv",index_col=0)

In [ ]:
data['Timestamp'] = pd.to_datetime(data['LastUpdatedDate'] + ' ' + data['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')
data = data.sort_values('Timestamp').reset_index(drop=True)

In [ ]:
data["VehicleType"][data["VehicleType"]=="cycle"]=1
data["VehicleType"][data["VehicleType"]=="bike"]=2
data["VehicleType"][data["VehicleType"]=="car"]=3
data["VehicleType"][data["VehicleType"]=="truck"]=4
data["TrafficConditionNearby"][data["TrafficConditionNearby"]=="low"]=1
data["TrafficConditionNearby"][data["TrafficConditionNearby"]=="average"]=2
data["TrafficConditionNearby"][data["TrafficConditionNearby"]=="high"]=3
data["TrafficConditionNearby"]=(data["TrafficConditionNearby"]).astype(int)
data["VehicleType"]=(data["VehicleType"]).astype(int)
data["IsSpecialDay"]=(data["IsSpecialDay"]).astype(int)
data["Occupancy"]=(data["Occupancy"]).astype(int)
data["Capacity"]=(data["Capacity"]).astype(int)
data["Occupancy_rate"]=data["Occupancy"]/data["Capacity"]

In [ ]:
class ParkingSchema2(pw.Schema):
  Timestamp : str
  VehicleType : int
  TrafficConditionNearby : int
  IsSpecialDay : int
  Occupancy_rate : float
  QueueLength : int

In [ ]:
data[["Timestamp","VehicleType","TrafficConditionNearby","IsSpecialDay","Occupancy_rate","QueueLength"]].to_csv("parking_stream2.csv",index=False)

In [ ]:
data_stream2=pw.demo.replay_csv("/content/parking_stream2.csv",schema=ParkingSchema2,input_rate=1800)

In [ ]:
fmt = "%Y-%m-%d %H:%M:%S"
data_with_time2 = data_stream2.with_columns(
    t = data_stream2.Timestamp.dt.strptime(fmt),
    day = data_stream2.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

In [ ]:
import datetime
delta_window2 = (
    data_with_time2.windowby(
        pw.this.t,
        instance=pw.this.day,
        window=pw.temporal.tumbling(duration=datetime.timedelta(minutes=30)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t=pw.this._pw_window_end,
        vehtype=pw.reducers.avg(pw.this.VehicleType),
        traffic=pw.reducers.max(pw.this.TrafficConditionNearby),
        special=pw.reducers.max(pw.this.IsSpecialDay),
        occ_rate_avg=pw.reducers.avg(pw.this.Occupancy_rate),
        queue=pw.reducers.max(pw.this.QueueLength),
    )
    .with_columns(
        price=pw.if_else(pw.this.queue>13,10*(1+4*(((pw.this.vehtype)/4)-(pw.this.traffic/3)**2+2*(pw.this.special)+(pw.this.occ_rate_avg)+((pw.this.queue)/15**0.5))/25),
              pw.if_else(pw.this.queue>10 ,10*(1+4*(((pw.this.vehtype)/4)-(pw.this.traffic/3)**2+2*(pw.this.special)+(pw.this.occ_rate_avg)+((pw.this.queue)/15**0.7))/25),
              pw.if_else(pw.this.queue>5 ,10*(1+3*(((pw.this.vehtype)/4)-(pw.this.traffic/3)**2+2*(pw.this.special)+(pw.this.occ_rate_avg)+((pw.this.queue)/15))/25),
              pw.if_else(pw.this.queue>3 ,10*(1+((pw.this.vehtype/4)+pw.this.occ_rate_avg+pw.this.special)/6),10)))),

    )

)

In [ ]:
pn.extension()
def price_plotter2(source):

    fig = bokeh.plotting.figure(
        height=400,
        width=1000,
        title="Pathway: Daily Parking Price_Model2",
        x_axis_type="datetime",
    )

    fig.line("t", "price", source=source, line_width=2, color="green")


    fig.circle("t", "price", source=source, size=6, color="navy")

    return fig
viz = delta_window2.plot(price_plotter2, sorting_col="t")
pn.Column(viz).servable()

In [ ]:
%%capture --no-display
pw.run()